# II.4 PLU and Cholesky factorisations

In this chapter we consider the following factorisations for square real invetible  matrices $A ∈ ℝ^{n × n}$:
1. The _LU factorisation_:
$ A = LU$ where $L$ is lower triangular and $U$ is upper triangular. This is equivalent to Gaussian elimination without pivoting,
so may not exist (e.g. if $A[1,1] = 0$).
1. The _PLU factorisation_:
$
A = P^⊤ LU
$
where $P$ is a permutation matrix, $L$ is lower triangular and $U$ is upper triangular. This is equivalent to Gaussian elimination with pivoting.
It always exists but may in extremely rare cases be unstable. 
2. For a square, _symmetric positive definite_ ($𝐱^⊤ A 𝐱 > 0$ for all $𝐱 ∈ ℝ^n$, $𝐱 ≠ 0$) 
matrix the LU decompostion has a special form which is called the _Cholesky factorisation_:
$
A = L L^⊤
$.
3. We also discuss timing and stability of the different factorisations.

In [1]:
using LinearAlgebra, Plots, BenchmarkTools

## 1. LU Factorisation

Just as Gram–Schmidt can be reinterpreted as a reduced QR factorisation,
Gaussian elimination  can be interpreted as an LU factorisation.



Consider the following set of $n × n$ lower triangular
matrices which equals identity apart from one-column:
$$
{\cal L}_j := \left\{I + \begin{bmatrix} 𝟎_j \\ 𝐥_j \end{bmatrix} 𝐞_j^⊤ : 𝐥_j ∈ ℝ^{n-j} \right\}
$$
where  $𝟎_j$ denotes the zero vector of length $j$. 
That is, if $L_j ∈ {\cal L}_j$ then it is equal to the identity matrix apart from in the $j$ th column:
$$
L_j = \begin{bmatrix}
        1 \\ & {⋱} \\ && 1 \\
                    && ℓ_{j+1,j} & 1 \\
                    && ⋮ && \dots \\
                    && ℓ_{n,j} & & & 1 \end{bmatrix} = 
$$

These satisify the following special properties:

**Proposition 1 (one-column lower triangular inverse)**
If $L_j ∈ {\cal L}_j$ then
$$
L_j^{-1}  = I - \begin{bmatrix} 𝟎_j \\ 𝐥_j \end{bmatrix} 𝐞_j^⊤ = \begin{bmatrix}
        1 \\ & ⋱ \\ && 1 \\
                    &&-ℓ_{j+1,j} & 1 \\
                    &&⋮ && \dots \\
                    &&-ℓ_{n,j} & & & 1 \end{bmatrix} ∈ {\cal L}_j.
$$


**Proposition 2 (one-column lower triangular multiplication)**
If $L_j ∈ {\cal L}_j$ and $L_k ∈ {\cal L}_k$ for $k ≥ j$ then
$$
L_j L_k =  I + \begin{bmatrix} 𝟎_j \\ 𝐥_j \end{bmatrix} 𝐞_j^⊤ +  \begin{bmatrix} 𝟎_k \\ 𝐥_k \end{bmatrix} 𝐞_k^⊤
$$


**Lemma 1 (one-column lower triangular with pivoting)**
If $σ$ is a permutation that leaves the first $j$
rows fixed (that is, $σ_ℓ = ℓ$ for $ℓ ≤ j$) and $L_j ∈ {\cal L}_j$ then
$$
P_σ L_j=  \tilde L_j P_σ
$$
where $\tilde L_j ∈ {\cal L}_j$.

**Proof**
Write
$$
P_σ = \begin{bmatrix} I_j \\ & P_τ \end{bmatrix}
$$
where $τ$ is the permutation with Cauchy notation
$$
\begin{pmatrix}
1 & ⋯ & n-j \\
σ_{j+1}-j & ⋯ & σ_n-j
\end{pmatrix}
$$
Then we have
$$
P_σ L_j = P_σ + \begin{bmatrix} 𝟎_j \\ P_τ 𝐥_j \end{bmatrix} 𝐞_j^⊤ =
\underbrace{(I +  \begin{bmatrix} 𝟎_j \\ P_τ 𝐥_j \end{bmatrix} 𝐞_j^⊤)}_{\tilde L_j} P_σ
$$
noting that $𝐞_j^⊤ P_σ = 𝐞_j^⊤$ (as $σ_j = j$). 
∎


Now consider standard Gaussian elimation where one row-reduces
to introduce zeros column-by-column. We will mimick the computation of the QR factorisation
to view this as a _triangular triangularisation_.


Consider the matrix
$$
L_1 = \begin{bmatrix} 1 \\ -{a_{21} \over a_{11}} & 1 \\ ⋮ &&⋱ \\
                -{a_{n1} \over a_{11}}  &&& 1
                \end{bmatrix} = I - \begin{bmatrix} 0 \\ {𝐚_1[2:n] \over 𝐚_1[1]} \end{bmatrix}  𝐞_1^⊤.
$$
We then have
$$
L_1 A =  \begin{bmatrix} u_{11} & u_{12} & ⋯ & u_{1n} \\ 
& 𝐚_2^1 & ⋯ & 𝐚_n^1   \end{bmatrix}
$$
where $𝐚_j^1 := (L_1 𝐚_j)[2:n]$ and $u_{1j} = a_{1j}$. But now consider
$$
L_2 := I - \begin{bmatrix} 0 \\ {𝐚_2^1[2:n-1] \over 𝐚_2^1[1]} \end{bmatrix}  𝐞_1^⊤.
$$
Then
$$
L_2 L_1 A = \begin{bmatrix} u_{11} & u_{12} & u_{13} & ⋯ & u_{1n} \\ 
    & u_{22} & u_{23} & ⋯ & u_{2n} \\
&& 𝐚_3^2 & ⋯ & 𝐚_n^2   \end{bmatrix}
$$
where 
$$
u_{2j} :=  (𝐚_j^1)[1] \qquad \hbox{and} \qquad 𝐚_j^2 := (L_2 𝐚_j^1)[2:n-1]
$$
Thus the first two columns are triangular. 

The inductive construction is again clear. If we define $𝐚_j^0 := 𝐚_j$ we
have the construction
$$
\begin{align*}
L_j &:= I - \begin{bmatrix} 𝟎_j \\ {𝐚_{j+1}^j[2:n-j] \over 𝐚_{j+1}^j[1]} \end{bmatrix} 𝐞_j^⊤ \\
𝐚_j^k &:= (L_k 𝐚_j^{k-1})[2:n-k+1]
 \\
u_{kj} &:= (L_k 𝐚_j^{k-1})[1]
\end{align*}
$$
Then
$$
L_{n-1} ⋯ L_1 A = \underbrace{\begin{bmatrix} 
u_{11} & ⋯ & u_{1n} \\ & ⋱ & ⋮\\
                                        && u_{nn}\end{bmatrix}}_U
$$
i.e.
$$
A = \underbrace{L_{1}^{-1} ⋯ L_{n-1}^{-1}}_L U.
$$

Writing
$$
L_j = I + \begin{bmatrix} 𝟎_j \\ \ell_{j+1,j} \\ ⋮ \\ \ell_{n,j} \end{bmatrix} 𝐞_j^⊤
$$
and using the properties of inversion and multiplication we therefore deduce
$$
L = \begin{bmatrix} 1 \\ -\ell_{21} & 1 \\
-\ell_{31} & -\ell_{32} & 1 \\
 ⋮ & ⋮ & ⋱ & ⋱ \\
    -\ell_{n1} & -\ell_{n2} & ⋯ & -\ell_{n,n-1} & 1
    \end{bmatrix}
$$




**Example 1 (computer)**
We will do a numerical example (by-hand is equivalent to Gaussian elimination).
The first lower triangular matrix is:

In [2]:
n = 4
A = randn(n,n)
L₁ = I -[0; A[2:end,1]/A[1,1]] * [1 zeros(1,n-1)]

4×4 Matrix{Float64}:
  1.0       -0.0  -0.0  -0.0
 -0.28594    1.0  -0.0  -0.0
 -0.286831  -0.0   1.0  -0.0
 -0.052006  -0.0  -0.0   1.0

Which indeed introduces zeros in the first column:

In [3]:
A₁ = L₁*A

4×4 Matrix{Float64}:
 -2.35039   0.545465   -0.595962   0.343301
  0.0       0.350853    0.963532  -0.545325
  0.0      -0.0781349   0.732061  -0.194058
  0.0       0.229514    0.745914   1.03162

Now we make the next lower triangular operator:

In [4]:
L₂ = I - [0; 0; A₁[3:end,2]/A₁[2,2]] * [0 1 zeros(1,n-2)]

4×4 Matrix{Float64}:
  1.0  -0.0       -0.0  -0.0
 -0.0   1.0       -0.0  -0.0
 -0.0   0.2227     1.0  -0.0
 -0.0  -0.654159  -0.0   1.0

So that

In [5]:
A₂ = L₂*L₁*A

4×4 Matrix{Float64}:
 -2.35039       0.545465     -0.595962   0.343301
  0.0           0.350853      0.963532  -0.545325
 -1.11022e-16  -1.38778e-17   0.94664   -0.315501
 -1.38778e-17   5.55112e-17   0.115612   1.38835

The last one is:

In [6]:
L₃ = I - [0; 0; 0; A₂[4:end,3]/A₂[3,3]] * [0 0 1 zeros(1,n-3)]

4×4 Matrix{Float64}:
  1.0  -0.0  -0.0       -0.0
 -0.0   1.0  -0.0       -0.0
 -0.0  -0.0   1.0       -0.0
 -0.0  -0.0  -0.122129   1.0

Giving us

In [7]:
U = L₃*L₂*L₁*A

4×4 Matrix{Float64}:
 -2.35039  0.545465     -0.595962   0.343301
  0.0      0.350853      0.963532  -0.545325
  0.0      0.0           0.94664   -0.315501
  0.0      2.77556e-17   0.0        1.42688

and

In [8]:
L = inv(L₁)*inv(L₂)*inv(L₃)

4×4 Matrix{Float64}:
 1.0        0.0       0.0       0.0
 0.28594    1.0       0.0       0.0
 0.286831  -0.2227    1.0       0.0
 0.052006   0.654159  0.122129  1.0

Note how the entries of `L` are indeed identical to the negative
lower entries of `L₁`, `L₂` and `L₃`.

**Example 2 (by-hand)**

Consider the matrix
$$
A = \begin{bmatrix} 1 & 1 & 1 \\
                    2 & 4 & 8 \\
                    1 & 4 & 9
                    \end{bmatrix}
$$
We have
$$
L_2 L_1 A = L_2 \begin{bmatrix}1 \\ 
                        -2 & 1 \\ -1 &  & 1 \end{bmatrix} \begin{bmatrix} 1 & 1 & 1 \\
                    2 & 4 & 8 \\
                    1 & 4 & 9
                    \end{bmatrix}
    = \begin{bmatrix}1 \\ & 1\\ & -{3 \over 2} & 1 
    \end{bmatrix} \begin{bmatrix} 1 & 1 & 1 \\
                         & 2 & 6 \\
                         & 3 & 8 \end{bmatrix}
    = \underbrace{\begin{bmatrix} 1 & 1 & 1 \\
                         & 2 & 6 \\
                         &  & -1 \end{bmatrix}}_U
$$
We then deduce $L$ by taking the negative of the lower 
entries of $L_1,L_2$:
$$
L = \begin{bmatrix} 1 \\ 2 & 1 \\ 1 &{3 \over 2} & 1
\end{bmatrix}.
$$


## 2. PLU Factorisation

We learned in first year linear algebra that if a diagonal entry is zero
whe1 doing Gaussian elimnation one has to _row pivot_. For stability, 
in implementation one _always_ pivots: swap the largest in magnitude entry for the entry on the diagonal.
In terms of a factorisation, this leads to 
$$
L_{n-1} P_{n-1} ⋯ P_2 L_1 P_1 A = U
$$
where $P_j$ is a permutation that leaves rows 1 through $j-1$ fixed,
and swaps row $j$ with a row $k \geq j$ whose entry is maximal in absolute value.

Thus we can deduce that 
$$
L_{n-1} P_{n-1} ⋯ P_2 L_1 P_1 = \underbrace{L_{n-1} \tilde L_{n-2} ⋯  \tilde L_1}_{L^{-1}}  \underbrace{P_{n-1} ⋯ P_2 P_1}_P.
$$
where the tilde denotes the combined actions of swapping the permutation and lower-triangular matrices, that is,
$$
P_{n-1}⋯ P_{j+1} L_j = \tilde L_j P_{n-1}⋯ P_{j+1}.
$$
where $\tilde L_j ∈ {\cal L}_j$.
The entries of $L$ are then again the negative of the entries below the diagonal of $L_{n-1}, \tilde L_{n-2}, \ldots,\tilde L_1$.


Writing
$$
\tilde L_j = I + \begin{bmatrix} 𝟎_j \\ \tilde \ell_{j+1,j} \\ ⋮ \\ \tilde \ell_{n,j} \end{bmatrix} 𝐞_j^⊤
$$
and using the properties of inversion and multiplication we therefore deduce
$$
L = \begin{bmatrix} 
1 \\ 
-\tilde \ell_{21} & 1 \\
-\tilde \ell_{31} & -\tilde \ell_{32} & 1 \\
 ⋮ & ⋮ & ⋱ & ⋱ \\
 -\tilde \ell_{n-1,1} & -\tilde \ell_{n-1,2} & ⋯ &  - \tilde \ell_{n-1,n-2} & 1 \\
    -\tilde \ell_{n1} & -\tilde \ell_{n2} & ⋯ &  - \tilde \ell_{n,n-2} &  -\ell_{n,n-1} & 1
\end{bmatrix}
$$



**Example 3**

Again we consider the matrix
$$
A = \begin{bmatrix} 1 & 1 & 1 \\
                    2 & 4 & 8 \\
                    1 & 4 & 9
                    \end{bmatrix}
$$
Even though $a_{11} = 1 ≠ 0$, we still pivot: placing 
the maximum entry on the diagonal to mitigate numerical errors.
That is, we first pivot and upper triangularise the first column:
$$
 L_1 P_1 A =  L_1\begin{bmatrix} 0 & 1 \\ 1 & 0 \\ && 1 \end{bmatrix}
\begin{bmatrix} 1 & 1 & 1 \\
                    2 & 4 & 8 \\
                    1 & 4 & 9
                    \end{bmatrix} = 
                     \begin{bmatrix}1 \\ -{1 \over 2} & 1 \\ -{1 \over 2} && 1 \end{bmatrix}
\begin{bmatrix} 2 & 4 & 8 \\
                1 & 1 & 1 \\
                    1 & 4 & 9
                    \end{bmatrix}
$$
We now pivot and upper triangularise the second column:
$$
  L_2 P_2 L_1 P_1 A = 
                    L_2 \begin{bmatrix}
                    1 \\ &0 & 1 \\ &1 & 0 \end{bmatrix}
\begin{bmatrix} 2 & 4 & 8 \\
                0 & -1 & -3 \\
                    0 & 2 & 5
                    \end{bmatrix}
                    = \begin{bmatrix} 1\\ & 1 \\ & {1 \over 2} & 1 \end{bmatrix}
\begin{bmatrix} 2 & 4 & 8 \\
                0 & 2 & 5 \\
                0 & -1 & -3 
                    \end{bmatrix} = 
                    \underbrace{\begin{bmatrix} 2 & 4 & 8 \\
                0 & 2 & 5 \\
                0 & 0 & -{1 \over 2}
                    \end{bmatrix}}_U
$$
We now move $P_2$ to the right:
$$
L_2 P_2 L_1 P_1 = \underbrace{\begin{bmatrix} 1\\ -{1 \over 2} & 1 \\  -{1 \over 2}  & {1 \over 2} & 1 \end{bmatrix}}_{L_2 \tilde L_1} \underbrace{\begin{bmatrix} 0 & 1 & 0 \\ 0 & 0 & 1 \\ 1 & 0 & 0 \end{bmatrix}}_P
$$
That is
$$
L = \begin{bmatrix} 1\\ {1 \over 2} & 1 \\  {1 \over 2}  & -{1 \over 2} & 1 \end{bmatrix}
$$

We see how this example is done on a computer:

In [9]:
A = [1 1 1;
     2 4 8;
     1 4 9]
L,U,σ = lu(A) # σ is a vector encoding the permutation

LU{Float64, Matrix{Float64}, Vector{Int64}}
L factor:
3×3 Matrix{Float64}:
 1.0   0.0  0.0
 0.5   1.0  0.0
 0.5  -0.5  1.0
U factor:
3×3 Matrix{Float64}:
 2.0  4.0   8.0
 0.0  2.0   5.0
 0.0  0.0  -0.5

The permutation is

In [10]:
σ

3-element Vector{Int64}:
 2
 3
 1

Thus to invert a system we can do:

In [11]:
b = randn(3)
U\(L\b[σ]) == A\b

true

Note the entries match exactly because this precisely what `\` is using.

## 3. Cholesky Factorisation

Cholesky Factorisation is a form of Gaussian elimination (without pivoting)
that exploits symmetry in the problem, resulting in a substantial speedup. 
It is only relevant for _symmetric positive definite_
matrices.

**Definition 1 (positive definite)** A square matrix $A ∈ ℝ^{n × n}$ is _positive definite_ if
for all $𝐱 ∈ ℝ^n, x ≠ 0$ we have
$$
𝐱^⊤ A 𝐱 > 0
$$

First we establish some basic properties of positive definite matrices:

**Proposition 3 (conj. pos. def.)** If  $A ∈ ℝ^{n × n}$ is positive definite and 
$V ∈ ℝ^{n × n}$ is non-singular then
$$
V^⊤ A V
$$
is positive definite.

**Proposition 4 (diag positivity)** If $A ∈ ℝ^{n × n}$ is positive definite
then its diagonal entries are positive: $a_{kk} > 0$.


**Theorem 1 (subslice pos. def.)** If $A ∈ ℝ^{n × n}$ is positive definite
and $𝐤 ∈ \{1,\ldots,n\}^m$ is a vector of $m$ integers where any integer appears only once,
 then $A[𝐤,𝐤] ∈ ℝ^{m × m}$ is also
positive definite.



We leave the proofs to the problem sheets. Here is the key result:


**Theorem 2 (Cholesky and sym. pos. def.)** A matrix $A$ is symmetric positive definite if and only if it has a Cholesky factorisation
$$
A = L L^⊤
$$
where the diagonals of $L$ are positive.

**Proof** If $A$ has a Cholesky factorisation it is symmetric ($A^⊤ = (L L^⊤)^⊤ = A$) and for $𝐱 ≠ 0$ we have
$$
𝐱^⊤ A 𝐱 = (L𝐱)^⊤ L 𝐱 = \|L𝐱\|^2 > 0
$$
where we use the fact that $L$ is non-singular.

For the other direction we will prove it by induction, with the $1 × 1$ case being trivial. 
Write
$$
A = \begin{bmatrix} α & 𝐯^⊤ \\
                    𝐯   & K
                    \end{bmatrix} = \underbrace{\begin{bmatrix} \sqrt{α} \\ 
                                    {𝐯 \over \sqrt{α}} & I \end{bmatrix}}_{L_1}
                                    \underbrace{\begin{bmatrix} 1  \\ & K - {𝐯 𝐯^⊤ \over α} \end{bmatrix}}_{A_1}
                                    \underbrace{\begin{bmatrix} \sqrt{α} & {𝐯^⊤ \over \sqrt{α}} \\
                                     & I \end{bmatrix}}_{L_1^⊤}.
$$
Note that $K - {𝐯 𝐯^⊤ \over α}$ is a subslice of $A_1 = L_1^{-1} A L_1^{-⊤}$, hence by the previous propositions is
itself symmetric positive definite. Thus we can write 
$$
K - {𝐯 𝐯^⊤ \over α} = \tilde L \tilde L^⊤
$$
and hence $A = L L^⊤$ for
$$
L= L_1 \begin{bmatrix}1 \\ & \tilde L \end{bmatrix}.
$$
satisfies $A = L L^⊤$.
∎


N1te hidden in this proof is a simple algorithm form computing the Cholesky factorisation.
We define
$$
\begin{align*}
A_1 &:= A \\
𝐯_k &:= A_k[2:n-k+1,1] \\
α_k &:= A_k[1,1] \\
A_{k+1} &:= A_k[2:n-k+1,2:n-k+1] - {𝐯_k 𝐯_k^⊤ \over α_k}.
\end{align*}
$$
Then
$$
L = \begin{bmatrix} \sqrt{α_1} \\
    {𝐯_1[1] \over \sqrt{α_1}}  &  \sqrt{α_2} \\
    {𝐯_1[2] \over \sqrt{α_1}}  & {𝐯_2[1] \over \sqrt{α_2}} &  \sqrt{α_2} \\
    ⋮ & ⋮ & ⋱ & ⋱ \\
    {𝐯_1[n-1] \over \sqrt{α_1}} &{𝐯_2[n-2] \over \sqrt{α_2}} & ⋯ & {𝐯_{n-1}[1] \over \sqrt{α_{n-1}}} & \sqrt{α_{n}}
    \end{bmatrix}
$$

This algorithm succeeds if and only if $A$ is symmetric positive definite.

**Example 4** Consider the matrix
$$
A_0 = A = \begin{bmatrix}
2 &1 &1 &1 \\
1 & 2 & 1 & 1 \\
1 & 1 & 2 & 1 \\
1 & 1 & 1 & 2
\end{bmatrix}
$$
Then
$$
A_1 = \begin{bmatrix}
2 &1 &1 \\
1 & 2 & 1 \\
1 & 1 & 2 
\end{bmatrix} - {1 \over 2} \begin{bmatrix} 1 \\ 1 \\ 1 \end{bmatrix} \begin{bmatrix} 1 & 1 & 1 \end{bmatrix} =
{1 \over 2} \begin{bmatrix}
3 & 1 & 1 \\
1 & 3 & 1 \\
1 & 1 & 3 
\end{bmatrix}
$$
Continuing, we have 
$$
A_2 = {1 \over 2} \left( \begin{bmatrix}
3 & 1 \\ 1 & 3
\end{bmatrix} - {1 \over 3} \begin{bmatrix} 1 \\ 1  \end{bmatrix} \begin{bmatrix} 1 & 1  \end{bmatrix}
\right)
= {1 \over 3} \begin{bmatrix} 4 & 1 \\ 1 & 4 \end{bmatrix}
$$
Finally
$$
A_3 = {5 \over 4}.
$$
Thus we get
$$
L= L_1 L_2 L_3 = \begin{bmatrix} \sqrt{2} \\ {1 \over \sqrt{2}} & {\sqrt{3} \over 2} \\ 
{1 \over \sqrt{2}} & {1 \over \sqrt 6} & {2 \over \sqrt{3}} \\
{1 \over \sqrt{2}} & {1 \over \sqrt 6} & {1 \over \sqrt{12}} & {\sqrt{5} \over 2}
\end{bmatrix}
$$


# 3. Timings and Stability

The different factorisations have trade-offs between speed and stability.
First we compare the speed of the different factorisations on a symmetric positive
definite matrix, from fastest to slowest:

In [12]:
n = 100
A = Symmetric(rand(n,n)) + 100I # shift by 10 ensures positivity
@btime cholesky(A);
@btime lu(A);
@btime qr(A);

  226.040 μs (3 allocations: 78.20 KiB)
  67.792 μs (4 allocations: 79.08 KiB)
  218.154 μs (7 allocations: 134.55 KiB)


On my machine, `cholesky` is ~1.5x faster than `lu`,  
which is ~2x faster than QR. 



In terms of stability, QR computed with Householder reflections
(and Cholesky for positive definite matrices) are stable, 
whereas LU is usually unstable (unless the matrix
is diagonally dominant). PLU is a very complicated story: in theory it is unstable,
but the set of matrices for which it is unstable is extremely small, so small one does not
normally run into them.

Here is an example matrix that is in this set.

In [13]:
function badmatrix(n)
    A = Matrix(1I, n, n)
    A[:,end] .= 1
    for j = 1:n-1
        A[j+1:end,j] .= -1
    end
    A
end
A =1badmatrix(5)

5×5 Matrix{Int64}:
  1   0   0   0  1
 -1   1   0   0  1
 -1  -1   1   0  1
 -1  -1  -1   1  1
 -1  -1  -1  -1  1

Note that pivoting will not occur (we do not pivot as the entries below the diagonal are the same magnitude as the diagonal), thus the PLU Factorisation is equivalent to an LU factorisation:

In [14]:
L,U = lu(A)

LU{Float64, Matrix{Float64}, Vector{Int64}}
L factor:
5×5 Matrix{Float64}:
  1.0   0.0   0.0   0.0  0.0
 -1.0   1.0   0.0   0.0  0.0
 -1.0  -1.0   1.0   0.0  0.0
 -1.0  -1.0  -1.0   1.0  0.0
 -1.0  -1.0  -1.0  -1.0  1.0
U factor:
5×5 Matrix{Float64}:
 1.0  0.0  0.0  0.0   1.0
 0.0  1.0  0.0  0.0   2.0
 0.0  0.0  1.0  0.0   4.0
 0.0  0.0  0.0  1.0   8.0
 0.0  0.0  0.0  0.0  16.0

But here we see an issue: the last column of `U` is growing exponentially fast! Thus when `n` is large
we get very large errors:

In [15]:
n = 100
b = randn(n)
A = badmatrix(n)
norm(A\b - qr(A)\b) # A \ b still uses lu

71.56223396519397

Note `qr` is completely fine:

In [16]:
norm(qr(A)\b - qr(big.(A)) \b) # roughly machine precision

5.831209081753154871879909088243956340161382800726458686474595140797341341552407e-15

Amazingly, PLU is fine if applied to a small perturbation of `A`:

In [17]:
ε = 0.000001
Aε = A .+ ε .* randn.()
norm(Aε \ b - qr(Aε) \ b) # Now it matches!

9.946474086751813e-15

The big _open problem_ in numerical linear algebra is to prove that the set of matrices
for which PLU fails has extremely small measure.